# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [325]:
import warnings
warnings.filterwarnings('ignore')

**1)** Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. Há linhas duplicadas?

Conte o número de valores missing por variável.

In [326]:
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

**2)** Conte o número de valores missing por variável.

In [327]:
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [268]:
sinasc.head(10)

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
5,1,2516500.0,110001,1,33,5.0,1 a 3 anos,621005.0,2.0,1.0,...,5,6,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
6,1,2516500.0,110001,1,37,1.0,8 a 11 anos,999992.0,1.0,0.0,...,5,7,ATIVO,MUNIC,Novo Horizonte do Oeste,Rondônia,-11.70974,-61.99439,230.0,843.446
7,1,2516500.0,110001,1,27,2.0,12 anos ou mais,514120.0,0.0,0.0,...,5,8,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
8,1,2679477.0,110001,1,20,5.0,8 a 11 anos,621005.0,0.0,1.0,...,2,9,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
9,1,2679477.0,110001,1,30,2.0,1 a 3 anos,999992.0,1.0,1.0,...,5,10,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


**3)** Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:  ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']  Refaça a contagem de valores missings.


In [328]:
dados = sinasc[['LOCNASC', 'IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO','GESTACAO','GRAVIDEZ','CONSULTAS','APGAR5']]
dados.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

**4)** Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.

In [329]:
 #dados['APGAR5'].dropna(inplace = True)

dados.dropna(subset=['APGAR5'], inplace=True) 

In [330]:
dados['APGAR5'].isna().sum()

0

**5)** observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.

In [331]:
#Perguntar pq nao func: dados[['ESTCIVMAE', 'CONSULTAS']].fillna(9, inplace=True)
dados[['ESTCIVMAE', 'CONSULTAS']] = dados[['ESTCIVMAE', 'CONSULTAS']].fillna(9)


In [332]:
dados[['ESTCIVMAE', 'CONSULTAS']].isna().sum()

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [333]:
dados[['ESTCIVMAE','CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [334]:
dados.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

**6)** Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.


In [335]:
#Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
dados['QTDFILVIVO'] = dados['QTDFILVIVO'].fillna(0)


In [336]:
dados['QTDFILVIVO'].isna().sum()

0

**7)** Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

In [337]:
dados.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [338]:
#Para as seguintes variaveis achei mais viavel preencher com N/I (Não informado) para os dados faltantes.
dados['ESCMAE']   = dados['ESCMAE'].fillna('N/I')
dados['GESTACAO'] = dados['GESTACAO'].fillna('N/I')
dados['GRAVIDEZ'] = dados['GRAVIDEZ'].fillna('N/I')


In [353]:
dados.loc[dados['APGAR5'] >=8].value_counts(['GESTACAO'])

GESTACAO           
37 a 41 semanas        22266
32 a 36 semanas         2134
N/I                     1189
42 semanas e mais        690
28 a 31 semanas          129
22 a 27 semanas           51
Menos de 22 semanas        4
Name: count, dtype: int64

Identificamos que para essa amostra de dados, a idade gestacional tem interferencia direta no APAGAR5.
Com destaque positivo na faixa entre 37 a 41 semanas e com destaque negativo após 42 semanas. 
Gestações abaixo de 37 semanas já é considerado prematuro e com risco de lesão cerebral (paralesia cerebral).

In [354]:
dados.loc[dados['APGAR5'] >=8].value_counts(['GRAVIDEZ'])

GRAVIDEZ       
Única              25926
Dupla                457
N/I                   75
Tríplice e mais        5
Name: count, dtype: int64

Identificamos que para essa amostra de dados, o tipo da gravidez tem interferencia direta no APAGAR5.
Quando a gravidez é Dupla ou Tríplice, já é considerado uma gravidez de risco.


**8)** O Apgar possui uma classificação indicando se o bebê passou por asfixia:

    Entre 8 e 10 está em uma faixa 'normal'.

    Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.

    Entre 4 e 5 significa 'asfixia moderada'.

    Entre 0 e 3 significa 'asfixia severa'.
    
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.
    

In [355]:
dados.loc[(dados['APGAR5'] >= 8), "Classificação APGAR5"] = 'Normal'
dados.loc[(dados['APGAR5'] >= 6.0) & (dados['APGAR5'] <= 7.0), "Classificação APGAR5"] = 'Asfixia leve'
dados.loc[(dados['APGAR5'] >= 4.0) & (dados['APGAR5'] <= 5.0), "Classificação APGAR5"] = 'Asfixia moderada'
dados.loc[(dados['APGAR5'] <= 3), "Classificação APGAR5"] = 'Asfixia severa'


In [356]:
dados.head(10)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Classificação APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,Normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,Normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal


**9)** Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [357]:
dados.rename(columns = str.lower)

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,classificação apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [358]:
dados.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'classificacao_apgar5']

In [359]:
dados.columns 

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'classificacao_apgar5'],
      dtype='object')

In [360]:
dados.head(10) 

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,classificacao_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,Normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,Normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
